In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.append("/Users/redstar/work/analysisHAL_miya/scripts/python_code-set/lib")
from common.io_data_bin          import input_bin_data_naxyzb, input_bin_data_nt
from common.statistics           import make_mean_err, make_JKsample
from fitting.fitfunc_type        import set_fitfunc_from_fname
from misc_QM.special_functions   import sph_harm_xyz, sph_harm_A1_xyz
from lattice.rotation_projection import rot_proj
from lattice.calc_laplacian      import lap, lap_4th_proc
from lattice.calc_potential      import calc_potential_t2, calc_potential_t2_num
from lattice.make_r_coordinates  import xyzdata_to_rdata, make_r_coord, make_rbin, divide_wave_L04, rbin_L04_separation
from lattice.misner_method       import Sph_harm_deco_misner
from sch_diffeq.solve_diffeq     import solve_sch_diff
from Tmatrix.convert_mat         import convert_TtoS
from Tmatrix.calc_phase_shift    import calc_phase_Sii, within_one
from sch_finiteV.solve_sch_finiteV import solve_sch_Fvol, solve_sch_Fvol_4th_prec
from multiprocessing             import Pool
# for fitting and phase shift calculation
from scipy.optimize import curve_fit
from sch_gauss_exp.solve_sch_GEM   import solve_sch_GEM

In [2]:
Lsize = 8
Tsize = 8

In [3]:
Nrelpt = 1
relpts = np.array([[(i%2)*2, (i//2)%2*2, (i//2)//2*2 ] for i in range(Nrelpt)])

In [4]:
ls

2pt_pi_1tave
4pt_correlatorhighexa_0
4pt_correlatorhighexa_1
4pt_correlatorhighexa_2
4pt_correlatorhighexa_3
4pt_correlatorhighexa_4
4pt_correlatorhighexa_5
4pt_correlatorhighexa_6
4pt_correlatorhighexa_7
4pt_correlatorhighrel_0
4pt_correlatorhighrel_1
4pt_correlatorhighrel_2
4pt_correlatorhighrel_3
4pt_correlatorhighrel_4
4pt_correlatorhighrel_5
4pt_correlatorhighrel_6
4pt_correlatorhighrel_7
4pt_correlatorlow_0
4pt_correlatorlow_1
4pt_correlatorlow_2
4pt_correlatorlow_3
4pt_correlatorlow_4
4pt_correlatorlow_5
4pt_correlatorlow_6
4pt_correlatorlow_7
NBS_exact8tave_x00y00z00
NBS_exact_afth1tave_x00y00z00
NBS_exact_bfth1tave_x00y00z00
NBS_lowmode_8tave
NBS_lowmode_aftth1tave1thr
NBS_lowmode_aftth1tave4thr
NBS_lowmode_befth1tave
NBS_lowmode_hoge1tave
NBS_rel8tave_x00y00z00
consistency_check_i1pipi_triangle_multithread.ipynb
consistency_check_i1pipi_triangle_multithread_2.ipynb


# low mode part 

In [5]:
wave_low_new = np.fromfile("NBS_lowmode_8tave",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [6]:
wave_low_old = np.array([np.fromfile("4pt_correlatorlow_%d"%it,'<c16') for it in range(Tsize)]).reshape(Tsize,Lsize,Lsize,Lsize)

In [7]:
wave_low_new

array([[[[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        

In [8]:
wave_low_old

array([[[[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
        

# high mode part (exact)

In [9]:
wave_highexa_new = np.fromfile("NBS_exact8tave_x00y00z00",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [10]:
wave_highexa_old = np.array([np.fromfile("4pt_correlatorhighexa_%d"%it,'<c16') for it in range(Tsize)]).reshape(Tsize,Lsize,Lsize,Lsize)

In [11]:
wave_highexa_new

array([[[[-7.45058060e-09+0.j, -2.98023224e-08+0.j,
           1.49011612e-08+0.j, ...,  1.49011612e-08+0.j,
          -3.72529030e-08+0.j,  2.23517418e-08+0.j],
         [ 7.45058060e-09+0.j,  7.45058060e-09+0.j,
          -5.21540642e-08+0.j, ..., -2.23517418e-08+0.j,
          -3.72529030e-08+0.j,  2.98023224e-08+0.j],
         [ 0.00000000e+00+0.j, -3.72529030e-08+0.j,
           0.00000000e+00+0.j, ..., -7.45058060e-09+0.j,
           7.45058060e-09+0.j, -3.72529030e-08+0.j],
         ...,
         [-4.47034836e-08+0.j, -2.98023224e-08+0.j,
           1.49011612e-08+0.j, ..., -4.47034836e-08+0.j,
          -2.23517418e-08+0.j,  2.98023224e-08+0.j],
         [ 6.70552254e-08+0.j, -1.04308128e-07+0.j,
          -9.68575478e-08+0.j, ..., -3.72529030e-08+0.j,
           1.49011612e-08+0.j,  0.00000000e+00+0.j],
         [-1.49011612e-08+0.j, -7.45058060e-09+0.j,
           2.98023224e-08+0.j, ..., -2.98023224e-08+0.j,
           1.49011612e-08+0.j,  3.72529030e-08+0.j]],

        [[ 1

In [12]:
wave_highexa_old

array([[[[-2.98023224e-08+0.j,  3.72529030e-08+0.j,
          -4.47034836e-08+0.j, ...,  0.00000000e+00+0.j,
           3.72529030e-08+0.j, -7.45058060e-09+0.j],
         [ 1.49011612e-08+0.j,  2.23517418e-08+0.j,
          -7.45058060e-09+0.j, ..., -2.23517418e-08+0.j,
           0.00000000e+00+0.j,  6.70552254e-08+0.j],
         [ 4.47034836e-08+0.j,  4.47034836e-08+0.j,
           0.00000000e+00+0.j, ..., -1.49011612e-08+0.j,
          -7.45058060e-09+0.j, -2.98023224e-08+0.j],
         ...,
         [ 0.00000000e+00+0.j, -7.45058060e-09+0.j,
           1.49011612e-08+0.j, ...,  0.00000000e+00+0.j,
           7.45058060e-09+0.j, -2.23517418e-08+0.j],
         [ 6.70552254e-08+0.j,  3.72529030e-08+0.j,
           2.23517418e-08+0.j, ...,  0.00000000e+00+0.j,
          -3.72529030e-08+0.j,  7.45058060e-09+0.j],
         [-3.72529030e-08+0.j, -2.23517418e-08+0.j,
           5.21540642e-08+0.j, ...,  5.96046448e-08+0.j,
          -2.23517418e-08+0.j,  5.96046448e-08+0.j]],

        [[-2

In [15]:
(wave_highexa_new - wave_highexa_old)

array([[[[ 2.23517418e-08+0.j, -6.70552254e-08+0.j,
           5.96046448e-08+0.j, ...,  1.49011612e-08+0.j,
          -7.45058060e-08+0.j,  2.98023224e-08+0.j],
         [-7.45058060e-09+0.j, -1.49011612e-08+0.j,
          -4.47034836e-08+0.j, ...,  0.00000000e+00+0.j,
          -3.72529030e-08+0.j, -3.72529030e-08+0.j],
         [-4.47034836e-08+0.j, -8.19563866e-08+0.j,
           0.00000000e+00+0.j, ...,  7.45058060e-09+0.j,
           1.49011612e-08+0.j, -7.45058060e-09+0.j],
         ...,
         [-4.47034836e-08+0.j, -2.23517418e-08+0.j,
           0.00000000e+00+0.j, ..., -4.47034836e-08+0.j,
          -2.98023224e-08+0.j,  5.21540642e-08+0.j],
         [ 0.00000000e+00+0.j, -1.41561031e-07+0.j,
          -1.19209290e-07+0.j, ..., -3.72529030e-08+0.j,
           5.21540642e-08+0.j, -7.45058060e-09+0.j],
         [ 2.23517418e-08+0.j,  1.49011612e-08+0.j,
          -2.23517418e-08+0.j, ..., -8.94069672e-08+0.j,
           3.72529030e-08+0.j, -2.23517418e-08+0.j]],

        [[ 3

# high mode part (relaxed)

In [16]:
wave_highrel_new = np.fromfile("NBS_rel8tave_x00y00z00",'<c16').reshape(Tsize, Lsize, Lsize, Lsize)

In [17]:
wave_highrel_old = np.array([np.fromfile("4pt_correlatorhighrel_%d"%it,'<c16') for it in range(Tsize)]).reshape(Tsize,Lsize,Lsize,Lsize)

In [18]:
wave_highrel_new

array([[[[-4.47034836e-08+0.j,  1.49011612e-08+0.j,
           1.49011612e-08+0.j, ...,  1.49011612e-08+0.j,
           1.49011612e-08+0.j,  7.45058060e-09+0.j],
         [-2.98023224e-08+0.j,  2.98023224e-08+0.j,
          -2.23517418e-08+0.j, ...,  3.72529030e-08+0.j,
           4.47034836e-08+0.j, -7.45058060e-09+0.j],
         [ 2.98023224e-08+0.j,  7.45058060e-08+0.j,
           4.47034836e-08+0.j, ..., -2.23517418e-08+0.j,
          -3.72529030e-08+0.j, -5.96046448e-08+0.j],
         ...,
         [-1.49011612e-08+0.j, -7.45058060e-09+0.j,
           1.49011612e-08+0.j, ..., -1.49011612e-08+0.j,
          -1.49011612e-08+0.j,  1.49011612e-08+0.j],
         [ 3.72529030e-08+0.j,  1.49011612e-08+0.j,
           7.45058060e-09+0.j, ...,  3.72529030e-08+0.j,
          -2.23517418e-08+0.j,  4.47034836e-08+0.j],
         [-3.72529030e-08+0.j,  2.98023224e-08+0.j,
           0.00000000e+00+0.j, ...,  5.21540642e-08+0.j,
           5.96046448e-08+0.j, -1.49011612e-08+0.j]],

        [[ 1

In [19]:
wave_highrel_old

array([[[[-7.45058060e-09+0.j,  1.49011612e-08+0.j,
          -4.47034836e-08+0.j, ...,  2.98023224e-08+0.j,
           7.45058060e-09+0.j, -6.70552254e-08+0.j],
         [ 3.72529030e-08+0.j, -5.96046448e-08+0.j,
           4.47034836e-08+0.j, ..., -4.47034836e-08+0.j,
          -2.23517418e-08+0.j,  2.98023224e-08+0.j],
         [ 2.98023224e-08+0.j,  7.45058060e-09+0.j,
           1.49011612e-08+0.j, ...,  2.98023224e-08+0.j,
          -5.21540642e-08+0.j,  7.45058060e-08+0.j],
         ...,
         [-1.49011612e-08+0.j,  0.00000000e+00+0.j,
           1.49011612e-08+0.j, ..., -7.45058060e-09+0.j,
          -7.45058060e-09+0.j, -8.94069672e-08+0.j],
         [-5.96046448e-08+0.j,  3.72529030e-08+0.j,
           7.45058060e-09+0.j, ..., -1.49011612e-08+0.j,
          -6.70552254e-08+0.j,  7.45058060e-08+0.j],
         [-1.49011612e-08+0.j, -4.47034836e-08+0.j,
           0.00000000e+00+0.j, ..., -2.98023224e-08+0.j,
          -7.45058060e-09+0.j, -2.23517418e-08+0.j]],

        [[-1

In [20]:
(wave_highrel_new - wave_highrel_old)

array([[[[-3.72529030e-08+0.j,  0.00000000e+00+0.j,
           5.96046448e-08+0.j, ..., -1.49011612e-08+0.j,
           7.45058060e-09+0.j,  7.45058060e-08+0.j],
         [-6.70552254e-08+0.j,  8.94069672e-08+0.j,
          -6.70552254e-08+0.j, ...,  8.19563866e-08+0.j,
           6.70552254e-08+0.j, -3.72529030e-08+0.j],
         [ 0.00000000e+00+0.j,  6.70552254e-08+0.j,
           2.98023224e-08+0.j, ..., -5.21540642e-08+0.j,
           1.49011612e-08+0.j, -1.34110451e-07+0.j],
         ...,
         [ 0.00000000e+00+0.j, -7.45058060e-09+0.j,
           0.00000000e+00+0.j, ..., -7.45058060e-09+0.j,
          -7.45058060e-09+0.j,  1.04308128e-07+0.j],
         [ 9.68575478e-08+0.j, -2.23517418e-08+0.j,
           0.00000000e+00+0.j, ...,  5.21540642e-08+0.j,
           4.47034836e-08+0.j, -2.98023224e-08+0.j],
         [-2.23517418e-08+0.j,  7.45058060e-08+0.j,
           0.00000000e+00+0.j, ...,  8.19563866e-08+0.j,
           6.70552254e-08+0.j,  7.45058060e-09+0.j]],

        [[ 2